In [28]:
#%pip freeze -r requirements.txt

In [ ]:
from dotenv import load_dotenv
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from PIL import Image, ImageDraw, ImageFont
import qrcode
import os


# Load Spotify API credentials
load_dotenv()
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

# Constants
GENRE = "Bollywood"
SIZE = 1063
ASSETS_DIR = f"assets/{GENRE}"
FONT_PATH = os.path.join(ASSETS_DIR, f"{GENRE}.ttf")
BG_PATH = os.path.join(ASSETS_DIR, f"{GENRE}.jpg")
GOLD = (220, 220, 220)
QR_SIZE = 600  # Size of the QR code
card_PATH = f"cards/{GENRE}"
os.makedirs(f'{card_PATH}/front', exist_ok=True)
os.makedirs(f'{card_PATH}/back', exist_ok=True)

def load_bg():
    bg = Image.open(BG_PATH).convert("RGB")
    return bg.resize((SIZE, SIZE))

def fit_and_draw_text(draw, text, base_font_size, y, max_width=SIZE - 100, split=False):
    font_size = base_font_size
    font = ImageFont.truetype(FONT_PATH, font_size)

    # Shrink font if too wide
    while draw.textbbox((0, 0), text, font=font)[2] > max_width and font_size > 20:
        font_size -= 4
        font = ImageFont.truetype(FONT_PATH, font_size)

    if split and draw.textbbox((0, 0), text, font=font)[2] > max_width:
        # Try splitting on a space near the middle
        words = text.split()
        if len(words) > 1:
            mid = len(words) // 2
            split_point = mid
            top = " ".join(words[:split_point])
            bottom = " ".join(words[split_point:])

            h_top = draw.textbbox((0, 0), top, font=font)[3]
            fit_and_draw_text(top, font_size, y)
            fit_and_draw_text(bottom, font_size, y + h_top + 10)
            return h_top * 2 + 10

    # Draw single line
    bbox = draw.textbbox((0, 0), text, font=font)
    w = bbox[2] - bbox[0]
    draw.text(((SIZE - w) // 2, y), text, font=font, fill=GOLD)
    return bbox[3] - bbox[1] + 10


def draw_text_card(title, artist, year, output_path):
    bg = load_bg()
    draw = ImageDraw.Draw(bg)

    y_cursor = SIZE // 3
    spacing = 20

    y_cursor += fit_and_draw_text(draw, f"\"{title}\"", 90, y_cursor, split=True)
    y_cursor += spacing
    y_cursor += fit_and_draw_text(draw, f"by {artist}", 80, y_cursor, split=True)
    y_cursor += spacing
    fit_and_draw_text(draw, f"{year}", 120, y_cursor)
    bg.save(output_path)
    print(f"✅ Saved front card: {output_path}")


def draw_qr_card(spotify_url, output_path):
    bg = load_bg()

    # Create QR code with custom parameters
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=1,
    )
    qr.add_data(spotify_url)
    qr.make(fit=True)

    # Generate QR code image with gray background
    img_qr = qr.make_image(fill_color="black", back_color=(180, 180, 180)).convert("RGB")
    qr_img = img_qr.resize((QR_SIZE, QR_SIZE))

    # Draw black bezel (optional)
    bezel_thickness = 10
    bezel_size = QR_SIZE + 2 * bezel_thickness
    qr_bg = Image.new("RGB", (bezel_size, bezel_size), "black")
    qr_bg.paste(qr_img, (bezel_thickness, bezel_thickness))

    # Paste QR centered on card
    qr_x = (SIZE - bezel_size) // 2
    qr_y = (SIZE - bezel_size) // 2
    bg.paste(qr_bg, (qr_x, qr_y))

    # Draw label below
    draw = ImageDraw.Draw(bg)
    font = ImageFont.truetype(FONT_PATH, 40)
    label = "SCAN TO LISTEN"
    bbox = draw.textbbox((0, 0), label, font=font)
    text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    text_x = (SIZE - text_w) // 2
    text_y = qr_y + bezel_size + 10  # 10px below QR

    draw.text((text_x, text_y), label, font=font, fill=GOLD)

    bg.save(output_path)
    print(f"✅ Saved back card: {output_path}")



def init_spotify():
    client_id = os.getenv("SPOTIFY_CLIENT_ID")
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
    auth = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    return spotipy.Spotify(client_credentials_manager=auth)

def get_song_info(sp, artist, title):
    query = f"track:{title} artist:{artist}"
    results = sp.search(q=query, type="track", limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        url = track['external_urls']['spotify']
        year = track['album']['release_date'].split('-')[0]
        return year, url
    return None, None


def parse_songs(path):
    songs = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or "-" not in line:
                continue
            artist, title = line.split("-", 1)
            songs.append((artist.strip(), title.strip()))
    return songs

def init_spotify():
    load_dotenv()  # loads SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET
    client_id = os.getenv("SPOTIFY_CLIENT_ID")
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
    auth = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    return spotipy.Spotify(client_credentials_manager=auth)

def get_song_info(sp, artist, title):
    query = f"track:{title} artist:{artist}"
    results = sp.search(q=query, type="track", limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        url = track['external_urls']['spotify']
        year = track['album']['release_date'].split('-')[0]
        return year, url
    return None, None


# Replace this block in your code
if __name__ == "__main__":
    os.makedirs("cards", exist_ok=True)

    songs = parse_songs(f"songs/{GENRE}.txt")
    sp = init_spotify()
    print(songs)
    for i, (artist, title) in enumerate(songs):
        print(f"🎵 {i+1:02} | {artist} - {title}")
        year, url = get_song_info(sp, artist, title)
    
        if not year or not url:
            print(f"   ❌ Failed to find on Spotify\n")
            continue

        safe_name = f"{i:02}_{re.sub(r'[^a-zA-Z0-9]', '_', artist)}_{re.sub(r'[^a-zA-Z0-9]', '_', title)}"
        front_path = f"{card_PATH}/front/{safe_name}.png"
        back_path = f"{card_PATH}/back/{safe_name}.png"

        draw_text_card(title, artist, year, front_path)
        draw_qr_card(url, back_path)

from PIL import Image
import glob
import re

def sort_key(path):
    # Sort using the numeric prefix (e.g. 00_Artist_Title.png)
    match = re.search(r'(\d+)_', os.path.basename(path))
    return int(match.group(1)) if match else 9999

def save_cards_as_pdf(output_pdf=f"{GENRE}_all_cards.pdf"):
    front_paths = sorted(glob.glob(f"cards/{GENRE}/front/*.png"), key=sort_key)
    back_paths = sorted(glob.glob(f"cards/{GENRE}/back/*.png"), key=sort_key)

    all_images = []

    for front, back in zip(front_paths, back_paths):
        front_img = Image.open(front).convert("RGB")
        back_img = Image.open(back).convert("RGB")
        all_images.extend([front_img, back_img])

    if all_images:
        all_images[0].save(
            output_pdf,
            save_all=True,
            append_images=all_images[1:],
            resolution=300,
            quality=95
        )
        print(f"✅ PDF saved: {output_pdf}")
    else:
        print("⚠️ No images found to save.")

# Call the function
save_cards_as_pdf(f"{GENRE}_all_cards.pdf")


[('Gauhar Jaan', 'Piya Bin Nahi Aavat Chain'), ('Kanan Devi', 'Main Ban Ki Chidiya'), ('Lata Mangeshkar', 'Aayega Aanewala'), ('Mukesh', 'Awaara Hoon'), ('Lata Mangeshkar', 'Yeh Zindagi Usi Ki Hai'), ('Mohammed Rafi', 'Baharon Phool Barsao'), ('Lata Mangeshkar', 'Ajeeb Dastan Hai Yeh'), ('Lata Mangeshkar', 'Chalte Chalte'), ('Kishore Kumar & Manna Dey', 'Yeh Dosti'), ('Kishore Kumar', 'Om Shanti Om'), ('Alka Yagnik', 'Ek Do Teen'), ('Udit Narayan & Sadhana Sargam', 'Pehla Nasha'), ('Lata Mangeshkar & Kumar Sanu', 'Tujhe Dekha To'), ('Sonu Nigam', 'Kal Ho Naa Ho'), ('Alisha Chinai, Shankar Mahadevan & Javed Ali', 'Kajra Re'), ('Arijit Singh', 'Tum Hi Ho'), ('Jubin Nautiyal & Asees Kaur', 'Raataan Lambiyan'), ('Ali Sethi & Shae Gill', 'Pasoori'), ('Arijit Singh & Shilpa Rao', 'Chaleya'), ('Shreya Ghoshal & Udit Narayan', 'Teri Ore'), ('Atif Aslam', 'Jeene Laga Hoon'), ('Shaan', 'Chand Sifarish'), ('Shankar Mahadevan', 'Breathless'), ('Kavita Krishnamurthy & Kumar Sanu', 'Dola Re Dola'), 

In [4]:
"""Here's the example from spotipy https://github.com/spotipy-dev/spotipy?tab=readme-ov-file#quick-start"""
from spotapi import Song

song = Song()
gen = song.paginate_songs("weezer")

# Paginates 100 songs at a time till there's no more
for batch in gen:
    for idx, item in enumerate(batch):
        print(idx, item['item']['data']['name'])
    
# ^ ONLY 6 LINES OF CODE

# Alternatively, you can query a specfic amount
songs = song.query_songs("weezer", limit=20)
data = songs["data"]["searchV2"]["tracksV2"]["items"]
for idx, item in enumerate(data):
    print(idx, item['item']['data']['name'])

0 Buddy Holly
1 Island In The Sun
2 Pink Triangle
3 My Name Is Jonas
4 Say It Ain't So
5 Say It Ain't So - Original Mix
6 I Wanna Be Adored - Remastered 2009
7 Undone - The Sweater Song
8 Cigarettes & Alcohol - Remastered
9 I Just Threw Out The Love Of My Dreams
10 Hash Pipe
11 Buddy Holly - 2024 Remaster
12 Beverly Hills
13 Only In Dreams
14 Pink Triangle
15 Think Fast (feat. Weezer)
16 Africa
17 Teenage Dirtbag
18 You Gave Your Love To Me Softly
19 Pork And Beans
20 My Name Is Jonas - 2024 Remaster
21 Lost in the Woods - Weezer Version
22 Undone - The Sweater Song - 2024 Remaster
23 Pork And Beans
24 The World Has Turned And Left Me Here
25 My Name Is Jonas
26 Buddy Holly
27 Girls & Boys - 2012 Remaster
28 Mr. Blue Sky
29 Lump
30 Enter Sandman
31 In The Garage
32 Staying Out For The Summer
33 Surf Wax America
34 Say It Ain't So - 2024 Remaster
35 Keep Fishin'
36 Elephant Stone - Remastered
37 Holiday
38 No One Else
39 I Do - Non-LP Version
40 Happy Together
41 Take on Me
42 Why Bothe

KeyboardInterrupt: 

In [8]:
from flask import Flask, request, jsonify
from spotapi import Login, Config, solver_clients, Song, Player

app = Flask(__name__)

# Replace with your actual credentials and API key
email = "your_email@example.com"
password = "your_spotify_password"
captcha_api_key = "your_captcha_api_key"

# Configure SpotAPI
cfg = Config(
    solver=solver_clients.Capsolver(captcha_api_key),
    logger=None
)

# Initialize and authenticate
login_instance = Login(cfg, password, email=email)
login_instance.login()

@app.route('/play', methods=['POST'])
def play_song():
    data = request.get_json()
    query = data.get('query')
    if not query:
        return jsonify({'error': 'No query provided'}), 400

    # Search for the song
    song = Song(client=login_instance.client)
    results = song.query_songs(query)
    tracks = results.get('tracks', {}).get('items', [])
    if not tracks:
        return jsonify({'error': 'No track found'}), 404

    # Get the top result's URI
    top_result = tracks[0]
    track_uri = top_result['uri']

    # Play the track
    player = Player(login_instance)
    player.play(track_uri)

    return jsonify({'message': f'Playing {top_result["name"]}'}), 200

if __name__ == '__main__':
    app.run(debug=True)


AttributeError: 'NoneType' object has no attribute 'attempt'